In [45]:
import glob, os
from tqdm import tqdm
# os.chdir("./quiztoolbox.com/")
confirmed_questions = set()
all_questions = set()
question_files = []
page_titles = []
sitename = "copyhomework"
for file in glob.glob("./" + sitename + ".com/**/*.html", recursive=True):
    with open(file, 'r') as file_reader:
        try:
            file_contents = file_reader.read()
#             print(file_contents)
            if "show answer" in file_contents.lower():
#                 print(file)
                question_files.append(file)
        except UnicodeDecodeError:
            pass
#             print("skipping", file)
# display(question_files)
print(len(question_files))

1971


In [46]:
class question():
    def __init__(self, text, tag=None):
        self.text = text
        if tag:
            self.tag = tag
        else:
            self.tag = ''
        
    def __lt__(self, other):
        return self.text < other.text
    
    def __eq__(self, other):
        return self.text == other.text and self.tag == other.tag
    
    def __hash__(self):
        return self.text.__hash__() + (self.tag.__hash__() if self.tag else 0)
    
    def __repr__(self):
        return str({'text': self.text, 'tag:': self.tag})

In [47]:
from bs4 import BeautifulSoup
from bs4 import UnicodeDammit
import html.parser as hp
from collections import Counter
import unidecode

In [48]:
def get_duplictes(lst):
    # https://stackoverflow.com/questions/52072381/how-to-print-only-the-duplicate-elements-in-python-list
    d =  Counter(lst)  # -> Counter({4: 3, 6: 2, 3: 1, 2: 1, 5: 1, 7: 1, 8: 1})
    res = [k for k, v in d.items() if v > 1]
    return res

In [ ]:
for file in tqdm(question_files):
#     file = question_files[-3]
#     print(file)
    with open(file, 'r') as file_reader:
    #     file_contents = UnicodeDammit(file_reader.read()).unicode_markup
        file_contents = hp.unescape(file_reader.read())

    parser = BeautifulSoup(file_contents, "lxml")
    # print(parser)
    tag = parser.find("meta",  property="og:description")
    if tag:
        tag = tag['content']
    headings = parser.find_all("div", class_="panel-heading")
    questions = []
    for h in headings:
        questions.append(question(unidecode.unidecode(h.text).strip(), tag))
    #     questions.append(h.get_text())

    questions = sorted(questions)
    all_questions.update(questions)
    found = get_duplictes(questions)
#     if len(found)==0:
#         print("missing questions on ", file)
    confirmed_questions.update(found)

 55%|█████▍    | 1082/1971 [00:13<00:10, 88.14it/s]

In [ ]:
print(len(confirmed_questions))
# display(sorted(confirmed_questions))
display(sorted(confirmed_questions, key=lambda q:q.tag))

In [ ]:
fake_questions = all_questions.difference(confirmed_questions)

In [ ]:
print(len(all_questions))
print(len(confirmed_questions))

In [ ]:
for s in confirmed_questions:
    if "honorlock".lower() in s.text.lower():
        print(s)

In [ ]:
import pickle

In [ ]:
file = open(sitename + '.all_q.txt', 'wb')
pickle.dump(all_questions, file)
file.close()

In [ ]:
file = open(sitename + '.confirmed_q.txt', 'wb')
pickle.dump(confirmed_questions, file)
file.close()